# Sire diaries: GPU-accelerated free energies

The [2023.4 release](https://sire.openbiosim.org) introduces support for [GPU-accelerated molecular dynamics alchemical free energy simulations](https://sire.openbiosim.org/tutorial/index_part06.html). This builds on the [integration with OpenMM]() introduced in the 2023.2 release, and the support for [trajectories](https://sire.openbiosim.org/versions/devel/cheatsheet/trajectory.html) and [units](https://sire.openbiosim.org/versions/devel/cheatsheet/units.html) introduced in 2023.3.

In this post, we will show how you can run a relative hydration free energy simulation, and then process the results using [alchemlyb](https://alchemlyb.readthedocs.io/en/latest/).

The concept of a [merged molecule](https://sire.openbiosim.org/tutorial/part06/01_merge.html) is central to the way that free energy calculations are implemented in sire. A merged molecule is one that represents both a “reference” state and a “perturbed” state. These are the two states that the free energy simulation will morph between, and for which the free energy difference will be calculated.

For example, here we have pre-prepared a merged molecule that represents the perturbation from ethane to methanol.

In [ ]:
import sire as sr
mols = sr.load(sr.expand(sr.tutorial_url, "merged_molecule.s3"))

This system contains a single merged molecule in a box of water. Merged molecules are idenfitied by the molecule property is_perturbable, which will be True. We can extract the merged molecule from this system using

In [ ]:
mol = mols["molecule property is_perturbable"]
print(mol)

Merged molecules contain two sets of the molecular properties; one that represents the reference state, and one that represents the perturbed state. These are identified by the 0 and 1 suffixes.

For example, the reference state atomic charges are in the "charge0" property;

In [ ]:
print(mol.property("charge0"))

while the perturbed state atomic charges are in the "charge1" property:

In [ ]:
print(mol.property("charge1"))

We can view the perturbed state by linking to its properties, e.g.

In [ ]:
mol = mol.perturbation().link_to_perturbed().commit()
mol["not element Xx"].view()

(noting to only view non-dummy atoms via `mol["not element Xx"]`)

We can view the reference state using

In [ ]:
mol = mol.perturbation().link_to_reference().commit()
mol.view()

A λ-coordinate is used to morph from the reference state (at λ=0) to the perturbed state (at λ=1). We can run dynamics at any λ-value just by passing this in as an argument to the [minimisation and dynamics functions](https://sire.openbiosim.org/tutorial/part06/05_free_energy_perturbation.html).

In [ ]:
mol = mol.minimisation(lambda_value=0.5).run().commit()

In [ ]:
d = mol.dynamics(lambda_value=0.5, temperature="25oC")
d.run("10ps")
print(d)

The next step is to calculate and store the energy during the trajectory of the molecules as a function of λ. We do this by creating an [EnergyTrajectory](https://sire.openbiosim.org/tutorial/part06/05_free_energy_perturbation.html). We do this by telling the dynamics simulation to save the energy periodically, via the `energy_frequency` argument.

In [ ]:
d = mol.dynamics(lambda_value=0.5, temperature="25oC")
d.run("10ps", energy_frequency="0.1ps")
print(d)

The energy trajectory is retrieved via the `energy_trajectory()` function.

In [ ]:
t = d.energy_trajectory()
print(t)

We calculate the free energy across λ by collecting and averaging the energy across many dynamics simulations run across λ. For this to work, the energy at neighbouring λ-values has to be evaluated in addition to the energy for the simulated λ-value. The `lambda_windows` argument lets us tell the simulation to evalute the energy at extra λ-values during the trajectory. 

For example, let's run the simulation at λ=0.5, while also calculating the energy at λ=0 and λ=1.

In [ ]:
d = mol.dynamics(lambda_value=0.5, temperature="25oC")
d.run("10ps", energy_frequency="0.1ps", lambda_windows=[0, 1])
print(d)

t0_5 = d.energy_trajectory()
print(t0_5)

To calculate a free energy, we would also need to run simulations at λ=0 and λ=1...

In [ ]:
d = mol.dynamics(lambda_value=0, temperature="25oC")
d.run("10ps", energy_frequency="0.1ps", lambda_windows=[0.5, 1])

t0_0 = d.energy_trajectory()

d = mol.dynamics(lambda_value=1, temperature="25oC")
d.run("10ps", energy_frequency="0.1ps", lambda_windows=[0, 0.5])

t1_0 = d.energy_trajectory()

[alchemlyb](https://alchemlyb.readthedocs.io/en/latest/) is a great tool that can process a table of energy values across λ, and compute a free energy with associated error.

First, we combine the three `EnergyTrajectory` objects to into a single [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) in [alchemlyb format](https://alchemlyb.readthedocs.io/en/latest/parsing.html#u-nk-standard-form).

In [ ]:
df = sr.morph.to_alchemlyb([t0_0, t0_5, t1_0])
print(df)

This DataFrame can be passed directly into alchemlyb to calculate the relative free energy using the [BAR method](https://alchemlyb.readthedocs.io/en/latest/estimators/alchemlyb.estimators.BAR.html#alchemlyb.estimators.BAR).

In [ ]:
from alchemlyb.estimators import BAR
b = BAR()
b.fit(df)
print(b.delta_f_.loc[0.00, 1.00])

One of the nice things about alchemlyb is that it is easy to use different free energy estimators to calculate the free energy. Let's try the more accurate [MBAR method](https://alchemlyb.readthedocs.io/en/latest/estimators/alchemlyb.estimators.MBAR.html#alchemlyb.estimators.MBAR).

In [ ]:
from alchemlyb.estimators import MBAR
b = MBAR()
b.fit(df)
print(b.delta_f_.loc[0.00, 1.00])

Not bad - both methods agree that the free energy calculated from these simulations is 3.0 kcal mol-1. This is only a rough estimate of the relative free energy, as we only used three λ-windows, and only ran very short (10 ps) dynamics simulations. A better estimate could be made by running more λ-windows and running longer dynamics simulations at each window.

[This script](https://sire.openbiosim.org/tutorial/part06/06_faster_fep.html#complete-example-script) let's you run longer simulations across more λ-windows, running the perturbation both for ethane to methanol in the gas phase and also in a box of water. Running 250 ps of dynamics across 21 evenly-spaced λ-windows gives a gas-phase relative free energy of 2.98 +/- 0.01 kcal mol-1. Our value calculated above using only three λ-windows and short simulations is remarkably close!

Combining this with the water-phase relative free energy of -3.30 +/- 0.02 kcal mol-1 gives a predicted relative hydration free energy of ethane and methanol of -6.28 +/- 0.02 kcal mol-1. This is in excellent agreement with [values computed with other codes](https://www.pure.ed.ac.uk/ws/portalfiles/portal/75900057/20181010_Michel_reprod.pdf).

Now that we know that the free energy code works, our focus now is checking accuracy and performance for protein-ligand relative and absolute binding free energy simulations. To do this, we've implemented support for [restraints](https://sire.openbiosim.org/tutorial/part06/03_restraints.html). These restraints can be [morphed using λ](https://sire.openbiosim.org/tutorial/part06/04_alchemical_restraints.html), via our new [LambdaSchedule objects](https://sire.openbiosim.org/tutorial/part06/02_alchemical_dynamics.html#controlling-perturbations-with-a-schedule). This schedule gives you control over how forcefield parameters and restraints are changed as a function of λ. For example, [here we show](https://sire.openbiosim.org/tutorial/part06/04_alchemical_restraints.html#alchemical-restraints) how you can set up a schedule that slowly turns on some restraints before morphing from the reference molecule to the perturbed molecule. Then, after the morph, these restraints are slowly turned off.

We think that this framework, and the concepts of merged molecules and λ-schedules gives a lot of power to users and developers of free energy simulations. We're looking forward to exploring this new functionality ourselves, and also seeing what everyone else in the community creates.

As with all of our diaries, if you want to try this yourself, please feel free connect to [try.openbiosim.org](https://try.openbiosim.org) and starting a notebook. You can download the notebook used to generate this post onto the server by running this command in one of the notebook code cells.

! wget https://github.com/OpenBioSim/posts/raw/main/sire/004_alchemy/alchemy.ipynb

Have a play and [let us know what you think](https://github.com/OpenBioSim/sire/issues).